In [41]:
import geopandas as gpd

# gdf = gpd.read_parquet('outputs/satellogic-earthview-items.parquet')
gdf = gpd.read_parquet('outputs/satellogic-earthview-items-with-matches.parquet')
gdf.head()

,json_path,zone,region,date,geometry,s2_matches,s1_matches
6292276,data/json/zone=51S/region=357067_6579845/date=...,51S,357067_6579845,2022-10-26,"POLYGON ((121.50829 -30.90962, 121.50835 -30.9...",[{'id': 'S2B_MSIL2A_20221106T015619_N0400_R117...,[]
5793006,data/json/zone=46N/region=374867_2457086/date=...,46N,374867_2457086,2022-12-25,"POLYGON ((91.7897 22.2112, 91.78967 22.21467, ...",[{'id': 'S2A_MSIL2A_20230108T042151_N0509_R090...,[{'id': 'S1A_IW_GRDH_1SDV_20230108T234835_2023...
3933716,data/json/zone=38N/region=601805_4299370/date=...,38N,601805_4299370,2022-10-05,"POLYGON ((46.17736 38.83375, 46.17741 38.83721...",[{'id': 'S2A_MSIL2A_20221020T073931_N0400_R092...,[{'id': 'S1A_IW_GRDH_1SDV_20221017T030130_2022...


In [52]:
sample = gdf.iloc[2]
sample

json_path     data/json/zone=38N/region=601805_4299370/date=...
zone                                                        38N
region                                           601805_4299370
date                                        2022-10-05 00:00:00
geometry      POLYGON ((46.177355725778 38.833747049158, 46....
s2_matches    [{'id': 'S2A_MSIL2A_20221020T073931_N0400_R092...
s1_matches    [{'id': 'S1A_IW_GRDH_1SDV_20221017T030130_2022...
Name: 3933716, dtype: object

In [53]:
from eotdl.access import find_sentinel_matches_by_centroid

centroid = sample.geometry.centroid
matches = find_sentinel_matches_by_centroid(centroid, sample.date, 30, 384, 384, 'sentinel-2-l2a')
# matches = sample.s2_matches
matches

[{'id': 'S2A_MSIL2A_20221020T073931_N0400_R092_T38SNH_20221020T110405',
  'properties': {'datetime': '2022-10-20T07:49:17Z', 'eo:cloud_cover': 63.35}},
 {'id': 'S2A_MSIL2A_20221020T073931_N0400_R092_T38SPH_20221020T110405',
  'properties': {'datetime': '2022-10-20T07:49:14Z', 'eo:cloud_cover': 61.05}},
 {'id': 'S2A_MSIL2A_20221020T073931_N0400_R092_T38SNJ_20221020T110405',
  'properties': {'datetime': '2022-10-20T07:49:02Z', 'eo:cloud_cover': 48.1}},
 {'id': 'S2A_MSIL2A_20221020T073931_N0400_R092_T38SPJ_20221020T110405',
  'properties': {'datetime': '2022-10-20T07:48:59Z', 'eo:cloud_cover': 64.92}},
 {'id': 'S2B_MSIL2A_20221018T074919_N0400_R135_T38SNH_20221018T092230',
  'properties': {'datetime': '2022-10-18T07:59:11Z', 'eo:cloud_cover': 13.75}},
 {'id': 'S2B_MSIL2A_20221018T074919_N0400_R135_T38SPH_20221018T092230',
  'properties': {'datetime': '2022-10-18T07:59:04Z', 'eo:cloud_cover': 14.87}},
 {'id': 'S2B_MSIL2A_20221018T074919_N0400_R135_T38SNJ_20221018T092230',
  'properties': {

In [54]:
matches = sample.s2_matches
matches

array([{'id': 'S2A_MSIL2A_20221020T073931_N0400_R092_T38SNH_20221020T110405', 'properties': {'datetime': '2022-10-20T07:49:17Z', 'eo:cloud_cover': 63.35}},
       {'id': 'S2A_MSIL2A_20221020T073931_N0400_R092_T38SPH_20221020T110405', 'properties': {'datetime': '2022-10-20T07:49:14Z', 'eo:cloud_cover': 61.05}},
       {'id': 'S2A_MSIL2A_20221020T073931_N0400_R092_T38SNJ_20221020T110405', 'properties': {'datetime': '2022-10-20T07:49:02Z', 'eo:cloud_cover': 48.1}},
       {'id': 'S2A_MSIL2A_20221020T073931_N0400_R092_T38SPJ_20221020T110405', 'properties': {'datetime': '2022-10-20T07:48:59Z', 'eo:cloud_cover': 64.92}},
       {'id': 'S2B_MSIL2A_20221018T074919_N0400_R135_T38SNH_20221018T092230', 'properties': {'datetime': '2022-10-18T07:59:11Z', 'eo:cloud_cover': 13.75}},
       {'id': 'S2B_MSIL2A_20221018T074919_N0400_R135_T38SPH_20221018T092230', 'properties': {'datetime': '2022-10-18T07:59:04Z', 'eo:cloud_cover': 14.87}},
       {'id': 'S2B_MSIL2A_20221018T074919_N0400_R135_T38SNJ_20221

In [55]:
from datetime import datetime

s2_matches_filtered = [r for r in matches if r['properties']['eo:cloud_cover'] <= 10]
closest_match = min(s2_matches_filtered, key=lambda x: abs(datetime.fromisoformat(x['properties']['datetime'].replace('Z','')) - sample.date))
s2_date = closest_match['properties']['datetime']
s2_date

'2022-10-05T07:48:55Z'

In [56]:
from eotdl.access import download_sentinel_imagery
from eotdl.tools import bbox_from_centroid

custom_bbox = bbox_from_centroid(x=centroid.y, y=centroid.x, pixel_size=10, width=384, height=384)
download_sentinel_imagery('outputs/kk', s2_date, custom_bbox, 'sentinel-2-l2a', 'kk')